## Receptive field

In [ ]:
import matplotlib.pyplot as plt
import itertools
import numpy as np
from matplotlib.cm import get_cmap
import matplotlib.patches as mpatches
from src.utils import save_figure


In [2]:
def analytical_solution(k, s, L):
    """
    Calculate the receptive field size for a multi-layer neural network.

    Parameters:
    k : list of int
        Layer kernel sizes [k_1, k_2, ..., k_L].
    s : list of int
        Layer strides [s_1, s_2, ..., s_L].
    L : int
        Number of layers.

    Returns:
    int
        The calculated receptive field size.
    """
    r = 1  # Initial receptive field size

    for l in range(1, L + 1):  # Iterate over each layer
        S = 1  # Reset cumulative stride
        for i in range(1, l):  # Compute the product of strides up to (l-1)
            S *= s[i - 1]  # Use zero-based indexing
        r += (k[l - 1] - 1) * S  # Update receptive field size for the layer

    return r


# Example usage:
k = [20,20,20,20,20,20]  # Kernel sizes for layers
s = [1,1,1,1,1,1]  # Strides for layers
L = len(k)  # Number of layers

receptive_field = analytical_solution(k, s, L)
print(f"Receptive field size: {receptive_field}")

Receptive field size: 115


In [62]:
k = [3,3,40]  # Kernel sizes for layers
s = [2,1,1]  # Strides for layers
L = len(k)  # Number of layers

receptive_field = analytical_solution(k, s, L)
print(f"Receptive field size: {receptive_field}")


Receptive field size: 85


In [15]:
k = [3,3,20,20,20,20,20,20]  # Kernel sizes for layers
s = [2,1,1,1,1,1,1,1]  # Strides for layers
L = len(k)  # Number of layers

receptive_field = analytical_solution(k, s, L)
print(f"Receptive field size: {receptive_field}")

Receptive field size: 235


In [22]:
k = [3,3,20,20,20,20,20,20]  # Kernel sizes for layers
s = [2,1,1,1,1,1,1,1]  # Strides for layers
L = len(k)  # Number of layers

receptive_field = analytical_solution(k, s, L)
print(f"Receptive field size: {receptive_field}")

Receptive field size: 235


In [13]:
def visualize_multiple_receptive_fields(configs, subplot_position, new_labels, title, bold_paths):
    """
    Visualize the receptive field growth for multiple configurations on the same plot.

    Parameters:
    configs : list of dict
        Each dict contains:
        - 'k': list of kernel sizes for each layer [k_1, k_2, ..., k_L].
        - 's': list of strides for each layer [s_1, s_2, ..., s_L].
        - 'label': str, label for the configuration.
    subplot_position : tuple
        Position of the subplot (rows, cols, index).
    title : str
        Title for the subplot.
    bold_paths : set
        Set of kernel sequences that should be bold.
    """
    plt.subplot(*subplot_position)
    
    common_color = "#1f77b4"  # Use the same color for all paths
    bold_linewidth = 2.0
    normal_linewidth = 0.7
    
    legend_handles = []  # Store legend entries

    for config in configs:
        k = tuple(config['k'])  # Convert to tuple for comparison
        s = config['s']
        L = len(k)  # Number of layers

        # Initialize receptive field
        r = 1  # Initial receptive field size
        receptive_fields = []  # Store RF for each layer
        layers = list(range(1, L + 1))  # Layer indices

        # Calculate receptive field size for each layer
        for l in range(1, L + 1):
            S = 1  # Cumulative stride
            for i in range(1, l):  # Compute cumulative stride
                S *= s[i - 1]
            r += (k[l - 1] - 1) * S
            receptive_fields.append(r)

        # Check if the path should be bold
        is_bold = k in bold_paths
        linewidth = bold_linewidth if is_bold else normal_linewidth
        color_value = "black" if is_bold else common_color
        alpha_value = 1 if is_bold else 0.5

        # Plot the progression
        plt.plot(layers, receptive_fields, linestyle="-", alpha=alpha_value, linewidth=linewidth, color=color_value)

        # Add to legend if bold
        if is_bold:
            legend_handles.append(mpatches.Patch(color="black", label=f"Kernel size path: {k}"))

    plt.axhline(y=134, color='red', linestyle='dashed')
    plt.text(0.9, 140, "AP length", fontsize=6, color='red')

    # Customize the plot
    plt.title(title)
    plt.ylabel("Receptive Field Size")
    plt.ylim(0, 730)
    plt.xticks(ticks=layers, labels=new_labels, rotation=75)
    plt.grid(True, linestyle="--", alpha=0.7)

    # Add legend for bold paths
    if legend_handles:
        plt.legend(handles=legend_handles, fontsize=5)


t = [3, 10, 20, 40]
c = list(itertools.combinations_with_replacement(t, 9))
unq = list(set(c))

# bold_paths = {
#     (3, 10, 20, 20, 20, 40),  # Example of a bold path
#     (20, 20, 20, 20, 20, 20),  # Another bold path
#     (40, 40, 40, 40, 40, 40)   # Another one
# }

# bold_paths_cnn = {
#     (3, 3, 3, 10, 20, 20, 20, 40),  # Example of a bold path
#     (3, 3, 20, 20, 20, 20, 20, 20),  # Another bold path
#     (3, 3, 40, 40, 40, 40, 40, 40)   # Another one
# }

bold_paths = {
}

bold_paths_cnn = {
}


configs_with_cnn = [
    {
        'k': np.append([3, 3], comb),
        's': [1, 2, 1, 1, 1, 1, 1, 1, 1,1,1,1],
        'label': f'Conv {comb[2]}x1: With initial CNN'
    }
    for comb in unq
]

configs = [
    {
        'k': comb,
        's': [1, 1, 1, 1, 1, 1, 1,1,1,1],
        'label': f'Conv {comb[-1]}x1: With initial CNN'
    }
    for comb in unq
]


visualize_multiple_receptive_fields(configs, 
                                    new_labels=["Conv1", "Conv2", "Conv3", "Conv4", "Conv5", "Conv6", "Conv7", "Conv8", "Conv9"],
                                    title="Receptive Field Growth for Unique Paths", 
                                    subplot_position=(1, 2, 1),
                                    bold_paths=bold_paths)

visualize_multiple_receptive_fields(configs_with_cnn, 
                                    new_labels=["Conv k=3, stride=2", "Conv k=3, stride=1", "Conv1", "Conv2", "Conv3", "Conv4", "Conv5", "Conv6", "Conv7", "Conv8", "Conv9"],
                                    title="Receptive Field Growth for Unique Paths with Initial CNN", 
                                    subplot_position=(1, 2, 2),
                                    bold_paths=bold_paths_cnn)

save_figure(name="receptive_field_with_cnn_d9", figdir='./plots', height=4, width=6)
